In [1]:
import librosa

import pandas as pd
import csv

import os
import sys
sys.path.insert(0, '../scripts')

from audio_overview import AudiOverview

import warnings
warnings.filterwarnings("ignore")

C:\Users\kiiru\.conda\envs\speech_recognition\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
metadata_path = '../rawdata/additional/Excerpts-Data/text/metadata.csv'

In [3]:
obj = AudiOverview()

In [4]:
def meta_data(path, sep="|"):
    df = pd.read_csv(path, sep=sep, header=None)
    df = df.drop([1], axis=1)
    df.columns=['filename', 'transcription']
    
    toDrop = []
    filepaths = []
    sample_rates = []
    duration = []
    
    for f in df['filename']:
        f_name = f.strip()
        #parent_folder =  f.split('_')[0]
        filepath = obj.audio_files(f_name)
        try:
            if os.path.isfile(filepath):
                audio, fs = librosa.load(filepath, sr=None)
                filepaths.append(filepath)
                sample_rates.append(fs)
                duration.append(float(len(audio)/fs))
            else:
                toDrop.append(f)
                #df = df[df.filename != f]
                continue
        except FilePathError:
            print("filepath not found")
    
    df = df[df.filename.isin(toDrop) == False]
    df['filepath'] = filepaths
    df['sample_rate'] = sample_rates
    df['duration'] = duration
    
    return df

In [5]:
df = meta_data(metadata_path)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1563 entries, 0 to 1569
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   filename       1563 non-null   object 
 1   transcription  1563 non-null   object 
 2   filepath       1563 non-null   object 
 3   sample_rate    1563 non-null   int64  
 4   duration       1563 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 73.3+ KB


In [7]:
df.head()

,filename,transcription,filepath,sample_rate,duration
0,KISWA-00001,Kenya yageuka taifa lililojaa matapeli ndani ...,../rawdata/additional/Excerpts-Data/wavs/KISWA...,22050,5.828209
1,KISWA-00002,NCHI YA WAKORA. Waziri arekebisha kosa lake l...,../rawdata/additional/Excerpts-Data/wavs/KISWA...,22050,5.398639
2,KISWA-00003,"WAZIRI wa TeknolojiaHabari na Mawasilino, Joe...",../rawdata/additional/Excerpts-Data/wavs/KISWA...,22050,12.118549
3,KISWA-00004,Bwana Mucheru alifutilia mbali uteuzi wa mare...,../rawdata/additional/Excerpts-Data/wavs/KISWA...,22050,10.275465
4,KISWA-00005,Bwana alipomteua Kochalle aliyeaga dunia mnam...,../rawdata/additional/Excerpts-Data/wavs/KISWA...,22050,6.292608


In [8]:
df.to_csv('../rawdata/additional/Excerpts-Data/text/cleaned_metadata.csv', index=False)